In [1]:
%pip install -r requirements.txt
# %restart_python


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
import mlflow
import pkg_resources

from mlflow.types.responses import ResponsesAgentRequest

from agent import LLM_ENDPOINT_NAME, AGENT

In [7]:
# input_example = {"input": [{"role": "user", "content": "How many Rs in strawberry?"}]}
input_example = ResponsesAgentRequest(input=[{"role": "user", "content": "How many Rs in strawberry?"}])

In [ ]:
with mlflow.start_run():
    AGENT.predict(input_example)

{"ts": "2025-08-19 13:22:20.165", "level": "ERROR", "logger": "pyspark.sql.connect.client.logging", "msg": "GRPC Error received", "context": {}, "exception": {"class": "_InactiveRpcError", "msg": "<_InactiveRpcError of RPC that terminated with:\n\tstatus = StatusCode.INTERNAL\n\tdetails = \"[CONFIG_NOT_AVAILABLE] Configuration spark.mlflow.modelRegistryUri is not available. SQLSTATE: 42K0I\"\n\tdebug_error_string = \"UNKNOWN:Error received from peer  {grpc_status:13, grpc_message:\"[CONFIG_NOT_AVAILABLE] Configuration spark.mlflow.modelRegistryUri is not available. SQLSTATE: 42K0I\"}\"\n>", "stacktrace": [{"class": null, "method": "config", "file": "/Users/stuart.lynn/Projects/u2m-experiment/agent/.venv/lib/python3.12/site-packages/pyspark/sql/connect/client/core.py", "line": "2055"}, {"class": null, "method": "__call__", "file": "/Users/stuart.lynn/Projects/u2m-experiment/agent/.venv/lib/python3.12/site-packages/grpc/_interceptor.py", "line": "277"}, {"class": null, "method": "_with_c

: 

In [0]:
resources = [mlflow.models.resources.DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME)]

In [0]:
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Workspace/Users/stuart.lynn@databricks.com/u2m-experiment")

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="agent",
        python_model="agent.py",
        input_example=input_example,
        resources=resources,
        extra_pip_requirements=[
            f"databricks-langchain=={pkg_resources.get_distribution('databricks-langchain').version}",
            f"langchain-core=={pkg_resources.get_distribution('langchain-core').version}",
            f"databricks-sdk=={pkg_resources.get_distribution('databricks-sdk').version}",
            f"mlflow[databricks]=={pkg_resources.get_distribution('mlflow').version}",
        ]
    )

In [0]:
loaded_model = mlflow.pyfunc.load_model(f"runs:/{logged_agent_info.run_id}/agent")
loaded_model.predict(input_example)

In [0]:
mlflow.models.predict(logged_agent_info.model_uri, input_example, env_manager="uv")

In [0]:
mlflow.models.predict(logged_agent_info.model_uri, input_example_extra_params, env_manager="uv")

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS stuart;
CREATE SCHEMA IF NOT EXISTS stuart.authtest;

In [0]:
mlflow.set_registry_uri("databricks-uc")

registered_model = mlflow.register_model(
    model_uri=logged_agent_info.model_uri,
    name="stuart.authtest.agent"
)

In [0]:
import databricks.agents

endpoint_name = "custom-agent-endpoint"

databricks.agents.deploy(
    model_name=registered_model.name,
    model_version=registered_model.version,
    scale_to_zero=True,
    endpoint_name=endpoint_name,
)